# 필요한 라이브러리 import

In [1]:
import csv
import pandas as pd
import os
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import warnings
import pickle
plt.rc('font',family='Malgun Gothic')
plt.rc('font',size=10)
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import load_model
import tensorflow as tf

warnings.filterwarnings(action='ignore')

In [2]:
import joblib

midfielder_model = joblib.load('./models/midfielder_lgbm_cluster.pkl')

    
premier_df = pd.read_csv('./datas/NEW_FM_DATA_MERGED/2021-2022/2021-2022-PremierLeague.csv')
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중 A/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,K Ps/90,태클 성공/90,드리블 성공/90,헤더 성공/90,Cr A/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [3]:
premier_df.rename(columns={'Cr C/A':'크로스 성공률','FA':'파울 획득','Int/90':'인터셉트/90','공중 A/90':'공중볼시도/90','K Ps/90':'키패스/90','Cr A/90':'크로스 시도/90'},inplace=True)
premier_df = premier_df.astype({'골':np.int64,'도움':np.int64,'파울 획득':np.int64,'태클 성공률':np.int64})
premier_df.head(1)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,공중볼시도/90,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기
0,Teemu Pukki,2021-2022,공격수,노리치,잉글랜드 프리미어 리그,7,2,0,7,11,...,7.28,100,30.64,26.0,0.8,0.88,0.98,3.39,0.58,0.94


In [4]:
midfielder_df = premier_df[premier_df['포지션']=='미드필더']

In [5]:
num_col = [midfielder_df.columns[i] for i,x in enumerate(midfielder_df.dtypes) if x == 'int64' or x == 'float64']
independent_value = set(num_col) - set(['평균획득승점/경기','출장시간','승','무','패','경기당 팀 득점/90','경기당 팀 실점/90'])
dependent_value = '평균획득승점/경기'

In [6]:
midfielder_df['예측PPG']=midfielder_model.predict(midfielder_df.loc[:,independent_value])

In [7]:
midfielder_df.sort_values(by='예측PPG',ascending=False).head(5)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,태클 성공률,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG
167,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,13,9,0,26,4,...,83,35.92,31.41,1.62,2.81,3.33,4.70,5.58,1.91,2.509206
166,Martin Ødegaard,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,15,11,0,26,7,...,75,47.66,41.45,1.65,1.07,1.37,2.97,4.16,1.89,2.459176
82,Bernardo Silva,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,15,9,0,26,11,...,78,40.24,35.38,1.82,1.42,2.39,3.34,4.92,1.85,2.441705
254,Mason Mount,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,17,6,0,36,11,...,82,40.77,35.28,1.91,1.71,1.57,4.32,6.23,2.02,2.413996
100,Bruno Fernandes,2021-2022,미드필더,맨 레드,잉글랜드 프리미어 리그,11,10,0,21,12,...,74,49.18,43.22,1.62,1.54,0.87,1.63,4.09,1.60,2.385953


In [8]:
midfielder_min = np.min(midfielder_df['예측PPG'].tolist())
midfielder_min

1.8140385214054027

In [9]:
midfielder_df['WAR'] = midfielder_df['예측PPG'].map(lambda x : x-midfielder_min+1)

In [10]:
midfielder_war = midfielder_df.sort_values(by='WAR',ascending=False).copy()
midfielder_war = midfielder_war.reset_index(drop=True)
midfielder_war.head(50)

,이름,시즌,포지션,구단,리그,골,도움,임대 기간,승,무,...,경기당 패스 시도/90,경기당 패스 성공/90,키패스/90,태클 성공/90,드리블 성공/90,헤더 성공/90,크로스 시도/90,평균획득승점/경기,예측PPG,WAR
0,Bukayo Saka,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,13,9,0,26,4,...,35.92,31.41,1.62,2.81,3.33,4.70,5.58,1.91,2.509206,1.695167
1,Martin Ødegaard,2021-2022,미드필더,아스날,잉글랜드 프리미어 리그,15,11,0,26,7,...,47.66,41.45,1.65,1.07,1.37,2.97,4.16,1.89,2.459176,1.645138
2,Bernardo Silva,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,15,9,0,26,11,...,40.24,35.38,1.82,1.42,2.39,3.34,4.92,1.85,2.441705,1.627667
3,Mason Mount,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,17,6,0,36,11,...,40.77,35.28,1.91,1.71,1.57,4.32,6.23,2.02,2.413996,1.599957
4,Bruno Fernandes,2021-2022,미드필더,맨 레드,잉글랜드 프리미어 리그,11,10,0,21,12,...,49.18,43.22,1.62,1.54,0.87,1.63,4.09,1.60,2.385953,1.571914
5,Hakim Ziyech,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,5,6,0,21,5,...,39.14,33.95,1.62,3.37,1.75,5.60,3.89,2.19,2.376617,1.562578
6,Kevin De Bruyne,2021-2022,미드필더,맨시티,잉글랜드 프리미어 리그,11,22,0,33,11,...,53.03,47.70,1.54,1.08,0.95,2.33,0.91,1.96,2.373945,1.559906
7,Harvey Barnes,2021-2022,미드필더,레스터,잉글랜드 프리미어 리그,14,7,0,22,14,...,34.44,30.42,1.65,2.16,3.19,4.05,5.62,1.45,2.366650,1.552612
8,Saúl,2021-2022,미드필더,첼시,잉글랜드 프리미어 리그,9,14,21/7/5 - 22/6/30,31,6,...,51.83,45.79,1.43,2.12,1.00,5.15,0.67,2.11,2.358203,1.544164
9,Alexis Mac Allister,2021-2022,미드필더,브라이튼,잉글랜드 프리미어 리그,8,3,0,11,9,...,42.12,35.32,1.63,1.00,0.98,4.26,5.79,1.11,2.342392,1.528354


In [23]:
midfielder_war[['이름','시즌','골','도움','경기당 패스 시도/90','경기당 패스 성공/90','WAR']].sort_values(by='WAR',ascending=False).head(40)

,이름,시즌,골,도움,경기당 패스 시도/90,경기당 패스 성공/90,WAR
0,Bukayo Saka,2021-2022,13,9,35.92,31.41,1.695167
1,Martin Ødegaard,2021-2022,15,11,47.66,41.45,1.645138
2,Bernardo Silva,2021-2022,15,9,40.24,35.38,1.627667
3,Mason Mount,2021-2022,17,6,40.77,35.28,1.599957
4,Bruno Fernandes,2021-2022,11,10,49.18,43.22,1.571914
5,Hakim Ziyech,2021-2022,5,6,39.14,33.95,1.562578
6,Kevin De Bruyne,2021-2022,11,22,53.03,47.70,1.559906
7,Harvey Barnes,2021-2022,14,7,34.44,30.42,1.552612
8,Saúl,2021-2022,9,14,51.83,45.79,1.544164
9,Alexis Mac Allister,2021-2022,8,3,42.12,35.32,1.528354


In [17]:
midfielder_war[['이름','시즌','골', '도움','WAR']].sort_values(by='골',ascending=False).reset_index(drop=True).query('이름 == "Bukayo Saka"')

,이름,시즌,골,도움,WAR
5,Bukayo Saka,2021-2022,13,9,1.695167


In [19]:
midfielder_war[['이름','시즌','골','경기당 패스 시도/90','경기당 패스 성공/90','도움','WAR']].sort_values(by='경기당 패스 시도/90',ascending=False).reset_index(drop=True).query('이름 == "Bukayo Saka"')

,이름,시즌,골,경기당 패스 시도/90,경기당 패스 성공/90,도움,WAR
130,Bukayo Saka,2021-2022,13,35.92,31.41,9,1.695167


In [20]:
midfielder_war[['이름','시즌','골','경기당 패스 시도/90','경기당 패스 성공/90','도움','WAR']].sort_values(by='경기당 패스 성공/90',ascending=False).reset_index(drop=True).query('이름 == "Bukayo Saka"')

,이름,시즌,골,경기당 패스 시도/90,경기당 패스 성공/90,도움,WAR
133,Bukayo Saka,2021-2022,13,35.92,31.41,9,1.695167


In [21]:
midfielder_war[['이름','시즌','골','경기당 패스 시도/90','경기당 패스 성공/90','도움','WAR']].sort_values(by='도움',ascending=False).reset_index(drop=True).query('이름 == "Bukayo Saka"')

,이름,시즌,골,경기당 패스 시도/90,경기당 패스 성공/90,도움,WAR
9,Bukayo Saka,2021-2022,13,35.92,31.41,9,1.695167
